# Investigating Stop Assignment

## Loading data

In [32]:
import requests
import pandas as pd
import numpy as np

### A single day's (05/24/2020) full data pulled from api

In [33]:
url = 'http://sfmta-test.eba-5ve9ec22.us-east-1.elasticbeanstalk.com/daily-general-json'

In [34]:
json = requests.get(url, params={'day': '2020-05-24'}).json()

In [35]:
# making df

df = pd.DataFrame(data=json)

In [36]:
# paring down to a couple of buses 
# (most reports and second-most reports) 
# on a single route to simplify
# and making sure we're sorted by time (stupid-check)

nbus = df[df['rid']=='NBUS']
nbus_highest = nbus[nbus['vid']==(nbus['vid'].value_counts().index[0])].sort_values('timestamp')
nbus_second = nbus[nbus['vid']==(nbus['vid'].value_counts().index[1])].sort_values('timestamp')

df = df.sort_values('timestamp')

In [37]:
# using stops gathered by Labs 22 for expediency

stops = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/sfmta-data-analysis-ds/master/deprecated_assets/datasets/route_info.csv')

In [38]:
nbus_stops = stops[stops['route_id']=='NBUS']

In [39]:
nbus_stops

,route_id,lat,lon,stopId,tag,title,dir
652,NBUS,37.77658,-122.39549,16695,6695,Townsend St & 4th St,Outbound
653,NBUS,37.77960,-122.38955,15235,5235,King St & 2nd St,Outbound
654,NBUS,37.78455,-122.38795,17447,7447,The Embarcadero & Brannan St,Outbound
655,NBUS,37.79108,-122.39010,14508,4508,The Embarcadero & Folsom St,Outbound
656,NBUS,37.79347,-122.39618,15669,5669,Market St & Drumm St,Outbound
...,...,...,...,...,...,...,...
716,NBUS,37.79257,-122.39702,15658,5658,Market St & Beale St,Inbound
717,NBUS,37.79056,-122.38990,14511,4511,The Embarcadero & Folsom St,Inbound
718,NBUS,37.78360,-122.38832,14531,4531,The Embarcadero & Townsend St,Inbound
719,NBUS,37.77980,-122.38994,15236,5236,King St & 2nd St,Inbound


## Engineering Probable Nearest Stop and Distance (For Confidence)

### Original helper function for wrangling

In [40]:
def wrangle_bus(df):
  """
  preps dataframe for a single bus
  gives accurate timestamps and naively calculates 
  dwell time as 1min per checkin with motion (kph <= 0)
  """
  times = df['timestamp'].values
  ages = df['age'].values
  df['adjusted_timestamp'] = [pd.Timestamp(times[x]) - pd.Timedelta(seconds=ages[x]) for 
                              x in range(len(df['timestamp']))]
  df['timestamp'] = [pd.Timestamp(times[x]) for x in range(len(df['timestamp']))]

  dwell_count = 0
  dwell_totals = []
  df['dwell'] = 0
  for x in df['kph']:
    if x <= 0:
      dwell_count += 1
      dwell_totals.append(dwell_count)
    elif x > 0:
      dwell_totals.append(0)
      dwell_count = 0
  df['dwell'] = [dwell_totals[x] for x in range(len(df))]

  return df

### Function to calculate nearest stop within $X$ km by projected euclidean distance

Significantly improved efficiency of execution time and time complexity. 

Also altered range for assigning stops to pick up some missed stops.

Code is also a lot easier to read without a million nested for loops.

Based off of the eyeball test code is doing a pretty good job of assigning stops\
appropriately. Dwell times are lining up closely with time at stops, and the progression of stops\
appears to be accurate in comparison to the route.

Relationship over with geopy / geodesic distance; [FCC projection][link] is bae now. 

The calculated distance is used to decide whether to assign a stop at a reported location or not;\
it also serves as a measure of confidence in the assigned stop.\
The smaller the distance, the more certain we are the bus is at or very near that stop.

Code below (or similar) should generalize - time/route/vehicle agnostic - and should generalize well to large amounts of data.\
I'm not a Big-O expert but if I'm understanding correctly our time-complexity\
should be much better than before.

[link]:https://www.govinfo.gov/content/pkg/CFR-2016-title47-vol4/pdf/CFR-2016-title47-vol4-sec73-208.pdf

In [41]:
def fcc_projection(loc1, loc2):
    """
    function to apply FCC recommended formulae
    for calculating distances on earth projected to a plane
    
    significantly faster computationally, negligible loss in accuracy
    
    Args: 
    loc1 - a tuple of lat/lon
    loc2 - a tuple of lat/lon
    """
    lat1, lat2 = loc1[0], loc2[0]
    lon1, lon2 = loc1[1], loc2[1]
    
    mean_lat = (lat1+lat2)/2
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    k1 = 111.13209 - 0.56605*np.cos(2*mean_lat) + .0012*np.cos(4*mean_lat)
    k2 = 111.41513*np.cos(mean_lat) - 0.09455*np.cos(3*mean_lat) + 0.00012*np.cos(5*mean_lat)
    
    distance = np.sqrt(np.square(k1*delta_lat) + np.square(k2*delta_lon))
    
    return distance

In [42]:
def assign_stop(df, stops):
    """
    applies basic wrangling function
    calculates nearest stop from reported location in km
    returns dataframe with reported location, 
    nearest stop (coords and name), and distance between

    tested with single buses on single routes on a single day;
    technically route/vehicle/time agnostic
    don't foresee any issues generalizing
    
    implements FCC projection formulae for calculating distance
    
    Args:
    df - dataframe of transit data, requires 'latitude', 'longitude' columns
    stops - datafram of stops data, requires 'lat', 'lon', 'title' columns
    """
    # original wrangling function for continuity
    wrangle_bus(df)

    # creating list of lat/lon dictionaries for stops and reported bus locations
    stop_lats = stops['lat'].values
    stop_lons = stops['lon'].values

    reported_lats = df['latitude'].values
    reported_lons = df['longitude'].values

    stop_points = [{'latitude': stop_lats[x], 'longitude': stop_lons[x]} 
                 for x in range(len(stops))]

    reported_points = [{'latitude': reported_lats[x], 
                      'longitude': reported_lons[x]} 
                     for x in range(len(df))]

    # to minimize possible overlap between probable stops
    # 500 ft as km
    # upper end of previous range for minimum distance between stops according to sfmta
    # this value seems good but could use more testing
    radius = .1524

    # dict to tuples to play nice with geopy
    stop_point_tuples = [tuple(stop_points[x].values()) 
                       for x in range(len(stop_points))]

    reported_point_tuples = [tuple(reported_points[x].values()) 
                           for x in range(len(reported_points))]

    df['reported_location'] = reported_point_tuples

    # generating ((lat/lon), distance) tuples for nearest stop within range
    # using FCC ellipsoidal earth projection
    distances = [{x: fcc_projection(location, x) 
                 for x in stop_point_tuples} 
                 for location in reported_point_tuples]
    
    # sorting for nearest stop
    distances_sorted = [{k: v for k, v in sorted(distances[x].items(), 
                                                key=lambda item: item[1])} 
                        for x in range(len(distances))]
    
    # creating list of nearest stops
    # nearest stop if nearest stop within radius, else None
    point_stops = [next(iter(distances_sorted[x].items())) 
                   if next(iter(distances_sorted[x].items()))[1] <= radius 
                   else None 
                   for x in range(len(distances_sorted))]
    
    # assigning stop name from stops table based on lat/lon from previous step
    stop_tuples = list(zip(stops['lat'], stops['lon']))
    stop_titles = [stops['title'].iloc[stop_tuples.index(stop[0])] 
                   if stop != None
                   else None 
                   for stop in point_stops]

    # pulling lat/lon and distance from tuples for df
    df['nearest_stop'] = [x[0] if x != None else None for x in point_stops]
    df['distance_in_km'] = [x[1] if x != None else None for x in point_stops]

    # pulling stop names from list for df
    df['stops'] = stop_titles

    # dropping columns of redundant information
    df = df.drop(columns=['age', 'rid', 'vid', 'latitude', 'longitude'])

    return df

## Results

### Full Day Test - Single Bus

In [44]:
%%timeit

assign_stop(nbus_highest, nbus_stops)

1.12 s ± 53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
nbus_highest_wrangled = assign_stop(nbus_highest, nbus_stops)

In [46]:
nbus_highest_wrangled[['adjusted_timestamp', 'kph', 'direction', 
                       'dwell', 'reported_location', 'stops', 
                       'nearest_stop', 'distance_in_km']].head(20)

,adjusted_timestamp,kph,direction,dwell,reported_location,stops,nearest_stop,distance_in_km
9458,2020-05-24 04:08:55,39,None,0,"(37.7595, -122.508)",Judah St & La Playa St,"(37.7602999, -122.50818000000001)",0.090679
9516,2020-05-24 04:09:55,8,None,0,"(37.7601, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.028759
9573,2020-05-24 04:10:43,0,None,1,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9630,2020-05-24 04:11:43,0,None,2,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9687,2020-05-24 04:12:42,0,None,3,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9746,2020-05-24 04:13:41,0,None,4,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9809,2020-05-24 04:14:41,0,None,5,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9872,2020-05-24 04:15:41,0,None,6,"(37.7602, -122.509)",Judah & La Playa St,"(37.7603599, -122.50900990000001)",0.017715
9935,2020-05-24 04:16:41,13,NBUS_I_F00,0,"(37.7604, -122.506)",Judah St & 46th Ave,"(37.7603899, -122.50606)",0.006760
10002,2020-05-24 04:17:40,32,NBUS_I_F00,0,"(37.7607, -122.5)",Judah St & 40th Ave,"(37.760740000000006, -122.49935990000002)",0.071259


### Full Day Test - All Lines, All Vehicles

In [ ]:
%%timeit

assign_stop(df, stops)

In [ ]:
df_wrangled = assign_stop(df, stops)

In [ ]:
df.head()